# Churn - Processing the Raw Data

### The origninal data files for churn example are:

### Files

1. **train.csv** size(44.5MB) about 1 million rows.

    the train set, containing the user ids and whether they have churned.

 + msno: user id
 + is_churn: This is the target variable. Churn is defined as whether the user did not continue the subscription within 30 days of expiration. is_churn = 1 means churn,is_churn = 0 means renewal.

2. **sample_submission_zero.csv** size(43.5MB) 970,960 rows

    the test set, containing the user ids, in the format that we expect you to submit

 + msno: user id
 + is_churn: This is what you will predict. Churn is defined as whether the user did not continue the subscription within 30 days of expiration. is_churn = 1 means churn,is_churn = 0 means renewal.

3. **transactions.csv** size(1.6GB) 21,547,746 rows

    transactions of users up until 2/28/2017.

 + msno: user id
 + payment_method_id: payment method
 + payment_plan_days: length of membership plan in days
 + plan_list_price: in New Taiwan Dollar (NTD)
 + actual_amount_paid: in New Taiwan Dollar (NTD)
 + is_auto_renew
 + transaction_date: format %Y%m%d
 + membership_expire_date: format %Y%m%d
 + is_cancel: whether or not the user canceled the membership in this transaction.

4. **user_logs.csv** size(28.4GB) 392,106,544 rows

    daily user logs describing listening behaviors of a user. Data collected until 2/28/2017.

 + msno: user id
 + date: format %Y%m%d
 + num_25: # of songs played less than 25% of the song length
 + num_50: # of songs played between 25% to 50% of the song length
 + num_75: # of songs played between 50% to 75% of of the song length
 + num_985: # of songs played between 75% to 98.5% of the song length
 + num_100: # of songs played over 98.5% of the song length
 + num_unq: # of unique songs played
 + total_secs: total seconds played

5. **members.csv** size(352MB) 5,116,194 rows

    user information. Note that not every user in the dataset is available.

 + msno
 + city
 + bd: age. Note: this column has outlier values ranging from -7000 to 2015, please use your judgement.
 + gender
 + registered_via: registration method
 + registration_init_time: format %Y%m%d
 + expiration_date: format %Y%m%d
 + Data Extraction Details


### Note:
One important information in the data extraction process is the definition of membership expiration date. Suppose we have a sequence for a user with the tuple of (transaction date, membership expiration date, and is_cancel):

(2017-01-01, 2017-02-28, false)

(2017-02-25, 0217-03-15, false)

(2017-04-30, 3017-05-20, false)

(data used for demo only, not included in competition dataset)

This user is included in the dataset since the expiration date falls within our time period. Since the subscription transaction is 30 days away from 2017-03-15, the previous expiration date, we will count this user as a churned user.

Let's consider a more complex example derive the last one, suppose now a user has the following transaction sequence

(2017-01-01, 2017-02-28, false)

(2017-02-25, 2017-04-03, false)

(2017-03-15, 2017-03-16, true)

(2017-04-01, 3017-06-30, false)

The above entries is quite typical for a user who changes his subscription plan. Entry 3 indicates that the membership expiration date is moved from 2017-04-03 back to 2017-03-16 due to the user making an active cancellation on the 15th. On April 1st, the user made a long term (two month subscription), which is 15 days after the "current" expiration date. So this user is not a churn user.

Now let's consider the a sequence that indicate the user does not falls in our scope of prediction

(2017-01-01, 2017-02-28, false)

(2017-02-25, 2017-04-03, false)

(2017-03-15, 2017-03-16, true)

(2017-03-18, 2017-04-02, false)

Note that even the 3rd entry has member ship expiration date falls in 2017-03-16, but the fourth entry extends the membership expiration date to 2017-04-02, not between 2017-03-01 and 2017-03-31, so we will not make a prediction for the user.

## Limitation of Resources

When handling large files, sometimes you are limited by resources: memory, disk space, time, etc.

Therefore, from the begining of a project that may involve large files, one need to develop a plan to address this problem.

### Memory usage

To optimize the usage of memory, followings are some useful tips:
1. Choose the "right" data types
2. When read in large files, larger than your sys memory, try read in by chunks (e.g., for csv files, you can choose to read in how many row or columns). Process chunks while reading.
3. recycle the object names, in python, names are shared views, a reference to say, data, but use with care, for some types, it may lead to confusion. 
4. use garbage collection.


### CPU usage

1. Use multi-core processing wherever possible
2. Use GPU wherever possible

In [1]:
import sys
import gc; gc.enable()# python garbage collection module, gc.enable() enables automatic garbage collection
import pandas as pd
import numpy as np
import time 
import datetime

### Read in member.csv

In [5]:
members=pd.read_csv('E:\\usu_classes\\fall_2019\\mis_6110_machine_learning\\data\\members.csv')
print('The shape of the data is {}.\nThe memory usage of each column of the data frame is \n{}\nThe total memory usage is {:6.2f} MB'.format(
       members.shape,members.memory_usage()/1024**2,members.memory_usage().sum()/1024**2))

The shape of the data is (5116194, 7).
The memory usage of each column of the data frame is 
Index                      0.000076
msno                      39.033463
city                      39.033463
bd                        39.033463
gender                    39.033463
registered_via            39.033463
registration_init_time    39.033463
expiration_date           39.033463
dtype: float64
The total memory usage is 273.23 MB


In [6]:
members.isnull().sum()

msno                            0
city                            0
bd                              0
gender                    3354778
registered_via                  0
registration_init_time          0
expiration_date                 0
dtype: int64

In [4]:
members.dtypes

msno                      object
city                       int64
bd                         int64
gender                    object
registered_via             int64
registration_init_time     int64
expiration_date            int64
dtype: object

In [7]:
import numpy as np
s=np.int8(0) # int8 has the range of [-127,127] 2**8/2-1
p=np.int64(0)

In [8]:
sys.getsizeof(s)

25

In [7]:
sys.getsizeof(p)

32

In [8]:
2**15

32768

### Re-read in member.csv

In [9]:
members=pd.read_csv('E:\\usu_classes\\fall_2019\\mis_6110_machine_learning\\data\\members.csv',
                    dtype={'city':'int8','bd':'int16','registered_via':'int8'},#specify data types
                    parse_dates=['registration_init_time','expiration_date'],#specify columns to be parsed
                    infer_datetime_format =True)

In [10]:
members.dtypes

msno                              object
city                                int8
bd                                 int16
gender                            object
registered_via                      int8
registration_init_time    datetime64[ns]
expiration_date           datetime64[ns]
dtype: object

In [11]:
print('The shape of the data is {}.\nThe memory usage of each column of the data frame is \n{}\nThe total memory usage is {:6.2f} MB'.format(
       members.shape,members.memory_usage()/1024**2,members.memory_usage().sum()/1024**2))

The shape of the data is (5116194, 7).
The memory usage of each column of the data frame is 
Index                      0.000076
msno                      39.033463
city                       4.879183
bd                         9.758366
gender                    39.033463
registered_via             4.879183
registration_init_time    39.033463
expiration_date           39.033463
dtype: float64
The total memory usage is 175.65 MB


#### Creat new columns for the year, month, and day for registration_init_time and expiration_date

In [12]:
members['registration_year']=members['registration_init_time'].dt.year.astype('int16')
members['registration_month']=members['registration_init_time'].dt.month.astype('int8')
members['registration_day']=members['registration_init_time'].dt.day.astype('int8')

members['expiration_year']=members['expiration_date'].dt.year.astype('int16')
members['expiration_month']=members['expiration_date'].dt.month.astype('int8')
members['expiration_day']=members['expiration_date'].dt.day.astype('int8')

In [13]:
members=members.drop(['registration_init_time','expiration_date'],axis=1)

In [14]:
print('The shape of the data is {}.\nThe memory usage of each column of the data frame is \n{}\nThe total memory usage is {:6.2f} MB'.format(
       members.shape,members.memory_usage()/1024**2,members.memory_usage().sum()/1024**2))

The shape of the data is (5116194, 11).
The memory usage of each column of the data frame is 
Index                  0.000076
msno                  39.033463
city                   4.879183
bd                     9.758366
gender                39.033463
registered_via         4.879183
registration_year      9.758366
registration_month     4.879183
registration_day       4.879183
expiration_year        9.758366
expiration_month       4.879183
expiration_day         4.879183
dtype: float64
The total memory usage is 136.62 MB


In [16]:
members.gender=members.gender.fillna('Unknown')

In [17]:
members.head()

,msno,city,bd,gender,registered_via,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,Unknown,9,2015,5,25,2015,5,26
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,Unknown,4,2016,12,21,2016,12,24
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,Unknown,4,2016,3,6,2016,3,9
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,2016,10,31,2016,11,7
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,Unknown,4,2017,2,2,2017,2,5


### Read in transaction.csv

#### Result if not specify dtypes:

`
The shape of the data is (21547746, 9).
The memory usage of each column of the data frame is 
Index                       0.000076
msno                      164.396255
payment_method_id         164.396255
payment_plan_days         164.396255
plan_list_price           164.396255
actual_amount_paid        164.396255
is_auto_renew             164.396255
transaction_date          164.396255
membership_expire_date    164.396255
is_cancel                 164.396255
dtype: float64
The total memory usage is 1479.57 MB
`

In [18]:
trans=pd.read_csv('E:\\usu_classes\\fall_2019\\mis_6110_machine_learning\\data\\transactions.csv',
                   dtype={'payment_method_id':'int8','payment_plan_days':'int16','is_auto_renew':'bool','is_cancel':'bool',
                         'payment_plan_days':'int16','plan_list_price':'int16','actual_amount_paid':'int16'},
                   parse_dates=['transaction_date','membership_expire_date'],#specify columns to be parsed
                   infer_datetime_format =True)

In [19]:
print('The shape of the data is {}.\nThe memory usage of each column of the data frame is \n{}\nThe total memory usage is {:6.2f} MB'.format(
       trans.shape,trans.memory_usage()/1024**2,trans.memory_usage().sum()/1024**2))

The shape of the data is (21547746, 9).
The memory usage of each column of the data frame is 
Index                       0.000076
msno                      164.396255
payment_method_id          20.549532
payment_plan_days          41.099064
plan_list_price            41.099064
actual_amount_paid         41.099064
is_auto_renew              20.549532
transaction_date          164.396255
membership_expire_date    164.396255
is_cancel                  20.549532
dtype: float64
The total memory usage is 678.13 MB


In [20]:
trans.dtypes

msno                              object
payment_method_id                   int8
payment_plan_days                  int16
plan_list_price                    int16
actual_amount_paid                 int16
is_auto_renew                       bool
transaction_date          datetime64[ns]
membership_expire_date    datetime64[ns]
is_cancel                           bool
dtype: object

In [21]:
trans['trans_year']=trans['transaction_date'].dt.year.astype('int16')
trans['trans_month']=trans['transaction_date'].dt.month.astype('int8')
trans['trans_day']=trans['transaction_date'].dt.day.astype('int8')

trans['trans_expiration_year']=trans['membership_expire_date'].dt.year.astype('int16')
trans['trans_expiration_month']=trans['membership_expire_date'].dt.month.astype('int8')
trans['trans_expiration_day']=trans['membership_expire_date'].dt.day.astype('int8')

#trans=trans.drop(['transaction_date','membership_expire_date'],axis=1)

In [22]:
print('The shape of the data is {}.\nThe memory usage of each column of the data frame is \n{}\nThe total memory usage is {:6.2f} MB'.format(
       trans.shape,trans.memory_usage()/1024**2,trans.memory_usage().sum()/1024**2))

The shape of the data is (21547746, 15).
The memory usage of each column of the data frame is 
Index                       0.000076
msno                      164.396255
payment_method_id          20.549532
payment_plan_days          41.099064
plan_list_price            41.099064
actual_amount_paid         41.099064
is_auto_renew              20.549532
transaction_date          164.396255
membership_expire_date    164.396255
is_cancel                  20.549532
trans_year                 41.099064
trans_month                20.549532
trans_day                  20.549532
trans_expiration_year      41.099064
trans_expiration_month     20.549532
trans_expiration_day       20.549532
dtype: float64
The total memory usage is 842.53 MB


## Read in train.csv

In [23]:
train=pd.read_csv('E:\\usu_classes\\fall_2019\\mis_6110_machine_learning\\data\\train.csv',dtype={'is_churn':'bool'})

In [24]:
train.dtypes

msno        object
is_churn      bool
dtype: object

In [25]:
train.shape

(992931, 2)

## Merge train, members

In [26]:
train=pd.merge(train,members,how='left',on='msno')
members=0

In [27]:
train.shape

(992931, 12)

In [28]:
train.head()

,msno,is_churn,city,bd,gender,registered_via,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,True,18.0,36.0,female,9.0,2005.0,4.0,6.0,2017.0,9.0,7.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,True,10.0,38.0,male,9.0,2005.0,4.0,7.0,2017.0,3.0,21.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,True,11.0,27.0,female,9.0,2005.0,10.0,16.0,2017.0,2.0,3.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,True,13.0,23.0,female,9.0,2005.0,11.0,2.0,2017.0,9.0,26.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,True,3.0,27.0,male,9.0,2005.0,12.0,28.0,2017.0,9.0,27.0


In [29]:
train.dtypes

msno                   object
is_churn                 bool
city                  float64
bd                    float64
gender                 object
registered_via        float64
registration_year     float64
registration_month    float64
registration_day      float64
expiration_year       float64
expiration_month      float64
expiration_day        float64
dtype: object

In [30]:
train.isnull().sum()

msno                       0
is_churn                   0
city                  116788
bd                    116788
gender                116788
registered_via        116788
registration_year     116788
registration_month    116788
registration_day      116788
expiration_year       116788
expiration_month      116788
expiration_day        116788
dtype: int64

In [31]:
train.memory_usage()/1024**2

Index                 7.575462
msno                  7.575462
is_churn              0.946933
city                  7.575462
bd                    7.575462
gender                7.575462
registered_via        7.575462
registration_year     7.575462
registration_month    7.575462
registration_day      7.575462
expiration_year       7.575462
expiration_month      7.575462
expiration_day        7.575462
dtype: float64

In [32]:
train['bd']=train['bd'].astype('float16')

In [33]:
train.memory_usage()/1024**2

Index                 7.575462
msno                  7.575462
is_churn              0.946933
city                  7.575462
bd                    1.893866
gender                7.575462
registered_via        7.575462
registration_year     7.575462
registration_month    7.575462
registration_day      7.575462
expiration_year       7.575462
expiration_month      7.575462
expiration_day        7.575462
dtype: float64

## Extract features from transaction

 + msno: user id
 + payment_method_id: payment method
 + payment_plan_days: length of membership plan in days
 + plan_list_price: in New Taiwan Dollar (NTD)
 + actual_amount_paid: in New Taiwan Dollar (NTD)
 + is_auto_renew
 + transaction_date: format %Y%m%d
 + membership_expire_date: format %Y%m%d
 + is_cancel: whether or not the user canceled the membership in this transaction.

### Features:

#### Marketing theory: RFM

RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries. [Wiki]https://en.wikipedia.org/wiki/RFM_(customer_value)

RFM stands for the three dimensions:
+ Recency – How recently did the customer purchase?
+ Frequency – How often do they purchase? 
+ Monetary Value – How much do they spend?

#### Application on transaction data:

**Variables for each row in transaction:**
1. Discount_value = plan_list_price - actual_amount_paid
2. Discount_y = 0 if Discount_value==0 else 1
3. Amount_per_day = actual_amount_paid / payment_plan_days (if days=0, then 0)

**Variables for each 'msno':**
3. Mean, std of above variables
3. number of transactions
3. number of is_cancel=1
4. number of discount / number of transactions
5. number of is_auto_renew / number of transactions
6. number of is_cancel / number of transactions

**Variables for the last transaction (for each 'msno'):**
1. Days from last transaction to 2017/02/28
2. Info about last transaction, all variables 
3. whether list price has been increased in the last **three** transactions

### Any other features you can think of?

### First, Keep only 'msno' that is in the train.

In [33]:
trans.memory_usage().sum()/1024**2

842.5308856964111

In [34]:
train2=pd.read_csv('..\\data\\churn\\train.csv',dtype={'is_churn':'bool'})

In [35]:
trans=pd.merge(train2,trans,how='left',on='msno')
trans.drop('is_churn',axis=1,inplace=True)
train2=[]

In [36]:
trans.memory_usage().sum()/1024**2

742.2201652526855

In [37]:
trans.shape

(15883148, 15)

In [38]:
trans.sort_values(by=['msno','transaction_date'],inplace=True)

In [39]:
trans.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,trans_month,trans_day,trans_expiration_year,trans_expiration_month,trans_expiration_day
4909146,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-11-16,2016-12-15,False,2016,11,16,2016,12,15
4909148,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-12-15,2017-01-15,False,2016,12,15,2017,1,15
4909145,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-01-15,2017-02-15,False,2017,1,15,2017,2,15
4909147,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15
10503284,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,True,2015-01-31,2015-03-19,False,2015,1,31,2015,3,19


**Variables for each row in transaction:**
1. Discount_value = plan_list_price - actual_amount_paid
2. Discount_y = 0 if Discount_value==0 else 1
3. Amount_per_day = actual_amount_paid / payment_plan_days (if days=0, then 0)

In [40]:
trans['discount_value'] = trans['plan_list_price'] - trans['actual_amount_paid']
trans['discount_y'] = (trans['discount_value']!=0)
trans['amount_per_day'] = trans['actual_amount_paid'] / trans['payment_plan_days']
trans['amount_per_day'].replace(np.inf,0,inplace=True)

In [41]:
trans.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,trans_month,trans_day,trans_expiration_year,trans_expiration_month,trans_expiration_day,discount_value,discount_y,amount_per_day
4909146,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-11-16,2016-12-15,False,2016,11,16,2016,12,15,0,False,3.300000
4909148,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-12-15,2017-01-15,False,2016,12,15,2017,1,15,0,False,3.300000
4909145,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-01-15,2017-02-15,False,2017,1,15,2017,2,15,0,False,3.300000
4909147,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,3.300000
10503284,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,True,2015-01-31,2015-03-19,False,2015,1,31,2015,3,19,0,False,4.806452


**Variables for each 'msno':**
3. Mean, std of above variables
3. number of transactions
3. number of is_cancel=1
4. number of discount / number of transactions
5. number of is_auto_renew / number of transactions
6. number of is_cancel / number of transactions

In sql, we could use group by, with pandas, there is also groupby method.

In [42]:
trans_gy = trans.groupby('msno') \
       .agg({'payment_method_id':'size', 'is_cancel':'sum','discount_y':'mean','is_auto_renew':'mean',
            'discount_value':'std'}) \
       .rename(columns={'payment_method_id':'trans_count','is_cancel':'cancel_count','discount_y':'discount_freq',
                        'is_auto_renew':'renew_freq','discount_value':'discount_std'}) \
       .reset_index()

In [43]:
trans_gy['cancel_freq']=trans_gy['cancel_count'] / trans_gy['trans_count']
trans_gy.head()

,msno,trans_count,cancel_count,discount_freq,renew_freq,discount_std,cancel_freq
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,0.0,0.000000,1.0,0.000000,0.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0.0,0.052632,1.0,34.182944,0.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,0.0,0.038462,1.0,29.221304,0.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,12,0.0,0.000000,1.0,0.000000,0.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,19,0.0,0.052632,1.0,34.182944,0.0


**Variables for the last transaction (for each 'msno'):**
1. Days from last transaction to 2017/02/28
2. Info about last transaction, all variables 
3. whether list price has been increased in the last **three** transactions

In [44]:
last_tran=trans.drop_duplicates(subset='msno',keep='last').copy()
last_tran['day_from_last']=(datetime.datetime(2017,2,28)-last_tran['transaction_date']).dt.days

In [45]:
#last_tran.drop('is_churn',axis=1,inplace=True)
last_tran.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,trans_month,trans_day,trans_expiration_year,trans_expiration_month,trans_expiration_day,discount_value,discount_y,amount_per_day,day_from_last
4909147,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,3.300000,13
10503290,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,True,2017-01-31,2017-03-19,False,2017,1,31,2017,3,19,0,False,4.966667,28
1414735,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,True,2017-02-26,2017-03-26,False,2017,2,26,2017,3,26,0,False,4.966667,2
6144648,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,4.966667,13
6625356,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30,149,149,True,2017-01-31,2017-03-23,False,2017,1,31,2017,3,23,0,False,4.966667,28


#### whether list price has been increased in the last three transactions

In [46]:
price_increase=trans.groupby('msno')['amount_per_day'].diff()

In [47]:
trans['price_increase']=price_increase

In [48]:
trans.head(30)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,trans_month,trans_day,trans_expiration_year,trans_expiration_month,trans_expiration_day,discount_value,discount_y,amount_per_day,price_increase
4909146,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-11-16,2016-12-15,False,2016,11,16,2016,12,15,0,False,3.300000,NaN
4909148,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2016-12-15,2017-01-15,False,2016,12,15,2017,1,15,0,False,3.300000,0.000000
4909145,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-01-15,2017-02-15,False,2017,1,15,2017,2,15,0,False,3.300000,0.000000
4909147,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,3.300000,0.000000
10503284,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,True,2015-01-31,2015-03-19,False,2015,1,31,2015,3,19,0,False,4.806452,NaN
10503289,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,True,2015-02-28,2015-04-19,False,2015,2,28,2015,4,19,0,False,4.806452,0.000000
10503286,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,True,2015-03-31,2015-05-19,False,2015,3,31,2015,5,19,0,False,4.806452,0.000000
10503288,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,0,0,149,True,2015-04-30,2015-06-19,False,2015,4,30,2015,6,19,-149,True,0.000000,-4.806452
10503293,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,True,2015-05-31,2015-07-19,False,2015,5,31,2015,7,19,0,False,4.966667,4.966667
10503294,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,True,2015-06-30,2015-08-19,False,2015,6,30,2015,8,19,0,False,4.966667,0.000000


In [49]:
price_increase=trans.groupby('msno').nth([-3,-2,-1]).reset_index()[['msno','price_increase']]

In [50]:
price_increase.head()

,msno,price_increase
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0


In [51]:
price_increase=price_increase.groupby('msno').agg({'price_increase':'sum'})

In [52]:
price_increase=price_increase.reset_index()

In [53]:
price_increase.head()

,msno,price_increase
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.0


### Check three dataframes again: trans_gy, last_tran,price_increase

In [54]:
trans_gy.head()

,msno,trans_count,cancel_count,discount_freq,renew_freq,discount_std,cancel_freq
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,0.0,0.000000,1.0,0.000000,0.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0.0,0.052632,1.0,34.182944,0.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,0.0,0.038462,1.0,29.221304,0.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,12,0.0,0.000000,1.0,0.000000,0.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,19,0.0,0.052632,1.0,34.182944,0.0


In [55]:
last_tran.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,trans_month,trans_day,trans_expiration_year,trans_expiration_month,trans_expiration_day,discount_value,discount_y,amount_per_day,day_from_last
4909147,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,3.300000,13
10503290,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,True,2017-01-31,2017-03-19,False,2017,1,31,2017,3,19,0,False,4.966667,28
1414735,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,True,2017-02-26,2017-03-26,False,2017,2,26,2017,3,26,0,False,4.966667,2
6144648,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,True,2017-02-15,2017-03-15,False,2017,2,15,2017,3,15,0,False,4.966667,13
6625356,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30,149,149,True,2017-01-31,2017-03-23,False,2017,1,31,2017,3,23,0,False,4.966667,28


In [56]:
print('{}\n{}\n{}'.format(trans_gy.shape, last_tran.shape,price_increase.shape))

(992931, 7)
(992931, 19)
(992931, 2)


### Merge the three dataframes

In [57]:
trans=pd.merge(last_tran,trans_gy,how='left',on='msno')

In [58]:
trans.shape

(992931, 25)

In [59]:
trans=pd.merge(trans,price_increase,how='left',on='msno')

In [60]:
trans.shape

(992931, 26)

In [61]:
trans.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_year,...,discount_y,amount_per_day,day_from_last,trans_count,cancel_count,discount_freq,renew_freq,discount_std,cancel_freq,price_increase
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,True,2017-02-15,2017-03-15,False,2017,...,False,3.300000,13,4,0.0,0.000000,1.0,0.000000,0.0,0.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,True,2017-01-31,2017-03-19,False,2017,...,False,4.966667,28,19,0.0,0.052632,1.0,34.182944,0.0,0.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,True,2017-02-26,2017-03-26,False,2017,...,False,4.966667,2,26,0.0,0.038462,1.0,29.221304,0.0,0.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,True,2017-02-15,2017-03-15,False,2017,...,False,4.966667,13,12,0.0,0.000000,1.0,0.000000,0.0,0.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30,149,149,True,2017-01-31,2017-03-23,False,2017,...,False,4.966667,28,19,0.0,0.052632,1.0,34.182944,0.0,0.0


In [62]:
train.shape

(992931, 12)

In [65]:
train=pd.merge(train,trans,how='left',on='msno')
trans=[]
last_tran=[]
trans_gy=[]
price_increase=[]

## Next step, feature extraction from user_log